## Column Description
<span style="font-size: 14px;">

**track_id**: The Spotify ID for the track

**artists**: The artists' names who performed the track. If there is more than one artist, they are separated by a ;

**album_name**: The album name in which the track appears

**track_name**: Name of the track

**popularity**: The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity.

**duration_ms**: The track length in milliseconds

**explicit**: Whether or not the track has explicit lyrics (true = yes it does; false = no it does not OR unknown)

**danceability**: Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable

**energy**: Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale

**key**: The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. If no key was detected, the value is -1

**loudness**: The overall loudness of a track in decibels (dB)

**mode**: Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0

**speechiness**: Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks

**acousticness**: A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic

**instrumentalness**: Predicts whether a track contains no vocals. "Ooh" and "aah" sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly "vocal". The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content

**liveness**: Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live

**valence**: A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry)

**tempo**: The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration

**time_signature**: An estimated time signature. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). The time signature ranges from 3 to 7 indicating time signatures of 3/4, to 7/4.

**track_genre**: The genre in which the track belongs
</span>

In [124]:
#import the modules we need
import pandas as pd
import numpy as np

#load the dataset as a pandas dataframe
df = pd.read_csv('dataset.csv', encoding='utf-8')

#remove null values
df.dropna(inplace=True)

#this data frame is a summary of the number of values in each column and their data type
summarydf = pd.DataFrame(columns=['Column', 'Data type', 'Size', 'Number of unique Values'])#, 'Num of Null Values'])


for loop_count, column in enumerate(df.columns):
    summarydf.loc[loop_count] = [column, df[column].dtype, df[column].size, df[column].nunique()]#, df[column].isnull().sum()]
    
summarydf.set_index('Column', inplace=True)

summarydf

#

,Data type,Size,Number of unique Values
Column,,,
Unnamed: 0,int64,113999,113999
track_id,object,113999,89740
artists,object,113999,31437
album_name,object,113999,46589
track_name,object,113999,73608
popularity,int64,113999,101
duration_ms,int64,113999,50696
explicit,bool,113999,2
danceability,float64,113999,1174


In [125]:
#Calculate total number of individual entries
print( 'Total number of data entries = '  + repr(df.columns.size)  + ' * ' + repr(df['Unnamed: 0'].size) + ' = ' + repr(df.columns.size *  df['Unnamed: 0'].size))

Total number of data entries = 21 * 113999 = 2393979


In [126]:
#total number of data entries -1 so that the index column is not factored into the calculation 
print( 'Total number of data entries (excluding index column) = '  + repr(df.columns.size - 1)  + ' * ' + repr(df['Unnamed: 0'].size) + ' = ' + repr((df.columns.size - 1) *  df['Unnamed: 0'].size))

Total number of data entries (excluding index column) = 20 * 113999 = 2279980


In [119]:
#function to check if a variable is a numpy float or int
def is_int_or_float(number):
    if type(number) == np.int64 or type(number) == np.int32 or type(number) == np.float64 or type(number) == np.float32:
        return True
    else:
        return False

In [120]:
#function to convert milliseconds into a string containing minutes and seconds, to improve readibility
def ms_to_minutes_seconds(milliseconds):
    if not is_int_or_float(milliseconds):
        return milliseconds
    total_seconds = milliseconds // 1000  
    minutes = total_seconds // 60         
    seconds = total_seconds % 60          
    return str(repr(int(minutes)) + "m and " + repr(int(seconds)) + "s")

In [121]:
#This data frame will contain descriptive statistics about the data
statsdf = pd.DataFrame(columns=['Column', 'Mean', 'Standard deviation', 'Variance', 'Min', 'Max', 'Range', 'Lower Quartile', 'Median', 'Upper Quartile'])

for loop_count, column in enumerate(df.columns):
    if loop_count == 0:
        continue #this is used to skip calculating values for the index row as it is pointless to perform calculations on it.
    
    if df[column].dtype == np.int64 or df[column].dtype == np.float64: #also ensure the columns we are performing calculations on only contain numerical values
        statsdf.loc[loop_count] = [column,
                                   df[column].mean(), 
                                   df[column].std(), 
                                   df[column].var(), 
                                   df[column].min(), 
                                   df[column].max(), 
                                   np.ptp(df[column]), 
                                   df[column].quantile(0.25), 
                                   df[column].median(),
                                   df[column].quantile(0.75)]
        
        if column == 'duration_ms':
            statsdf.loc[loop_count + 1] = statsdf.loc[loop_count].apply(ms_to_minutes_seconds)
            statsdf.loc[loop_count + 1, 'Column'] = 'duration minutes and seconds'


statsdf.set_index('Column', inplace=True)
statsdf

,Mean,Standard deviation,Variance,Min,Max,Range,Lower Quartile,Median,Upper Quartile
Column,,,,,,,,,
popularity,33.238827,22.304959,497.511199,0,100,100,17.0,35.0,50.0
duration_ms,228031.153387,107296.057716,11512444001.445007,8586,5237295,5228709,174066.0,212906.0,261506.0
duration minutes and seconds,3m and 48s,1m and 47s,191874m and 4s,0m and 8s,87m and 17s,87m and 8s,2m and 54s,3m and 32s,4m and 21s
danceability,0.566801,0.173543,0.030117,0.0,0.985,0.985,0.456,0.58,0.695
energy,0.641383,0.25153,0.063267,0.0,1.0,1.0,0.472,0.685,0.854
key,5.309126,3.559999,12.673594,0,11,11,2.0,5.0,8.0
loudness,-8.25895,5.029357,25.294436,-49.531,4.532,54.063,-10.013,-7.004,-5.003
mode,0.637558,0.480708,0.23108,0,1,1,0.0,1.0,1.0
speechiness,0.084652,0.105733,0.011179,0.0,0.965,0.965,0.0359,0.0489,0.0845


In [127]:
#TODO: group by genre
#    : compare popularity by genre
#    : compare most popular songs
#    : Analyse qualities which lead to greater popularity